# Cutting Cloud Costs with Infrastructure Automation (Part IV - Preparing and Training Code on AWS w/ Drop Out - MNIST)
I said in the last post that I was ready to try this on our face. I'm totally not ready - I'm just missing one last thing I wanted to try. TFlearn gives us the option of easily including _**drop out**_ in our fully connected layers of the NN, and I wanted to explore that really quickly before moving on because I've read about it and it seems super easily to implement.

## Drop Out
Drop out is a simple regularization concept in NNs. When we implement drop out, we're telling the NN to basically pretend like a few of the neurons in the fully connected layer don't exist. This forces the other neurons (who have not been dropped out) to remodel their weights / thresholds to work with different neurons in every iteration of training.

![](http://cs231n.github.io/assets/nn2/dropout.jpeg)

I like to think of this method employing the same logic that subsampling does in tree techniques like random forests. When we subsample in random forest, we choose only a subset of samples and / or features to train on. This basically forces different features to explicitly coexist with some features and prevent the coexistence with other features to get a bit more perspective on which features are truly more important.

Dropping out within NNs also force features (or linear combinations of features, depending on how deep we get into our fully connected layers) to work nice with other features to see which really end up making an impact in every situation before completing our model training.

Dropping out in TFlearn is super simple. There's literally a _**dropout**_ function where you tell it what percentage of the neurons to keep in every iteration and TFlearn will perform this for you by dropping our random neurons.

It's actually been in the code this whole time because sentdex used dropout in his model and I just copied sentdex, but I've been commenting it out. All the code below is _**the exact same as the previous post**_ except the dropout line when building our NN.

In [1]:
# Install tflearn
import os
os.system("sudo pip install tflearn")

0

In [2]:
# TFlearn libraries
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
import tflearn.datasets.mnist as mnist

# General purpose libraries
import matplotlib.pyplot as plt
import numpy as np
import math

In [3]:
# Extract data from mnist.load_data()
x, y, x_test, y_test = mnist.load_data(one_hot = True)

Extracting mnist/train-images-idx3-ubyte.gz
Extracting mnist/train-labels-idx1-ubyte.gz
Extracting mnist/t10k-images-idx3-ubyte.gz
Extracting mnist/t10k-labels-idx1-ubyte.gz


In [4]:
# Reshape x
x_reshaped = x.reshape([-1, 28, 28, 1])
print 'x_reshaped has the shape {}'.format(x_reshaped.shape)

x_reshaped has the shape (55000, 28, 28, 1)


In [5]:
# Reshape x_test
x_test_reshaped = x_test.reshape([-1, 28, 28, 1])
print 'x_test_reshaped has the shape {}'.format(x_test_reshaped.shape)

x_test_reshaped has the shape (10000, 28, 28, 1)


In [6]:
# sentdex's code to build the neural net using tflearn
#   Input layer --> conv layer w/ max pooling --> conv layer w/ max pooling --> fully connected layer --> output layer
convnet = input_data(shape = [None, 28, 28, 1], name = 'input')

convnet = conv_2d(convnet, 32, 2, activation = 'relu')
convnet = max_pool_2d(convnet, 2)

convnet = conv_2d(convnet, 64, 2, activation = 'relu')
convnet = max_pool_2d(convnet, 2)

convnet = fully_connected(convnet, 1024, activation = 'relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 10, activation = 'softmax')
convnet = regression(convnet, optimizer = 'sgd', learning_rate = 0.01, loss = 'categorical_crossentropy', name = 'targets')

In [7]:
model = tflearn.DNN(convnet)
model.fit(
    {'input': x_reshaped}, 
    {'targets': y}, 
    n_epoch = 5, 
    validation_set = ({'input': x_test_reshaped}, {'targets': y_test}), 
    snapshot_step = 500, 
    show_metric = True
)

Training Step: 4299  | total loss: 0.11463 | time: 11.579s
| SGD | epoch: 005 | loss: 0.11463 - acc: 0.9676 -- iter: 54976/55000
Training Step: 4300  | total loss: 0.11163 | time: 12.897s
| SGD | epoch: 005 | loss: 0.11163 - acc: 0.9693 | val_loss: 0.09128 - val_acc: 0.9733 -- iter: 55000/55000
--


## Results
Nice. We're at 97.3% and we've gained like 0.7%, which is actually a big deal in this application! At this level, we're basically trying to squeeze every 1/10th of a percent we can get haha.